In [10]:
# pip install langchain-pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone, ServerlessSpec
from langchain_community.vectorstores import FAISS, chroma, pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone as PC
import os
from dotenv import load_dotenv

In [ ]:
# from pinecone import Pinecone
# pc = Pinecone(api_key="your api_key")  

 
# index_name="your index name"
# index = pc.Index(index_name)  
# for i, t in zip(range(len(text_chunks)), text_chunks):
#    query_result = embeddings.embed_query(t.page_content)
#    index.upsert(
#    vectors=[
#         {
#             "id": str(i),  # Convert i to a string
#             "values": query_result, 
#             "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
#         }
#     ],
#     namespace="real" 
# )

#     index.describe_index_stats()

In [ ]:
# from langchain.vectorstores import Pinecone as PC

# docs_chunks = [t.page_content for t in text_chunks]
# pinecone_index = PC.from_texts(
#     docs_chunks,
#     hf,
#     index_name='your-index-name'
# )

In [2]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [3]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [4]:
extracted_data = load_pdf(r"C:\Users\DEEPAK YADAV\Downloads\Medical Chatbot\data/")
text_chunks = text_split(extracted_data)


In [5]:
docs_chunks = [t.page_content for t in text_chunks]

In [25]:
# docs_chunks

In [6]:
#download embedding model
# from sentence_transformers import SentenceTransformer
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings()

C:\Users\DEEPAK YADAV\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
# calculated from the input array of text chunks
emb_vectors = embeddings.embed_documents(docs_chunks) 

In [11]:
load_dotenv()
KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=KEY) 

In [12]:
index_name = "database"
index = pc.Index(index_name)

In [14]:
document = PC.from_texts(docs_chunks, embeddings, index_name=index_name)

In [15]:
document

In [16]:
document=PC.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=document.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Definition\nAllergy tests indicate a person’s allergic sensitivity\nto commonly encountered environmental substances.\nPurpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-ganisms and particles, like pollen or dust, by producingspecific proteins called antibodies that are capable ofbinding to identifying molecules, or antigens, on theforeign organisms. This reaction between antibody andantigen sets off a series of reactions designed to protectthe body from infection. Sometimes, this same series ofreactions is triggered by harmless, everyday substances.This is the condition known as allergy, and the offend-ing substance is called an allergen. Common inhaledallergens include pollen, dust, and insect parts from tinyhouse mites. Common food allergens include nuts, fish,and milk.\nAllergic reactions involve a special set of cells in'), Document(page_content='Definition\nAllergy tests indicate a person’

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
model_name_or_path = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin"

In [24]:
from huggingface_hub import hf_hub_download

In [25]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-7b-chat.ggmlv3.q4_0.bin:   0%|          | 0.00/3.79G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':1024,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=document.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])